# Playing with convolutions in TensorFlow
TensorFlow has great support for convolutional layers. The most popular one is tf.nn.conv2d.
```python
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
Input: Batch size x Height x Width x Channels
Filter: Height x Width x Input Channels x Output Channels (e.g. [5, 5, 3, 64])
Strides: 4 element 1-D tensor, strides in each direction (often [1, 1, 1, 1] or [1, 2, 2, 1])
Padding: 'SAME' or 'VALID'
Data_format: default to NHWC
```
![convolution operation effect](./pic/convolution.png)
There are also several other built-in convolutional operations.
## Convnet on MNIST
-----
For MNIST, we will be using two convolutional layers, each followed by a relu and a maxpool layers, and one fully connected layer. 
![mnist convolution version structure](./pic/mnist_convolution_version.png)
### Variable scope
Since we’ll be dealing with multiple layers, it’s important to introduce variable scope. Think of a variable scope something similar to a namespace. A variable name ‘weights’ in variable scope ‘conv1’ will become ‘conv1-weights’. The common practice is to create a variable scope for each layer, so that if you have variable ‘weights’ in both convolution layer 1 and convolution layer 2, there won’t be any name clash.
In variable scope, we don’t create variable using tf.Variable, but instead use tf.get_variable()
```python
tf.get_variable(<name>, <shape>, <initializer>)
```
If a variable with that name already exists in that variable scope, we use that variable. If a variable with that name doesn’t already exists in that variable scope, TensorFlow creates a new variable. This setup makes it really easy to share variables across architecture. This will come in extremely handy when you build complex models and you need to share large sets of variables. Variable scopes help you initialize all of them in one place.
Nodes in the same variable scope will be grouped together, and therefore you don’t have to use name scope any more. To declare a variable scope, you do it the same way you do name scope:
```python
with tf.variable_scope('conv1') as scope:
```
For example:
```python
with tf.variable_scope('conv1') as scope:
    w = tf.get_variable('weights', [5, 5, 1, 32])
    b = tf.get_variable('biases', [32], initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(images, w, strides=[1, 1, 1, 1], padding='SAME')
    conv1 = tf.nn.relu(conv + b, name=scope.name)
with tf.variable_scope('conv2') as scope:
    w = tf.get_variable('weights', [5, 5, 32, 64])
    b = tf.get_variable('biases', [64], initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(conv1, w, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + b, name=scope.name)
```

 Please refer to the [official documentation](https://www.tensorflow.org/api_docs/) for more information.
 
 ## Code
 ----
Below code is simple to help understand convolution interfaces of tensorflow. It only reached 0.92 accuracy, not good enough...

In [2]:
from __future__ import division
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

WORK_DIR = os.path.abspath(os.curdir)
LEARNING_RATE = 1e-4
BATCH_SIZE = 50

class MnistConvnet:
    def __init__(self, learning_rate, batch_size, data_path="./data/mnist"):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.data = input_data.read_data_sets(os.path.join(WORK_DIR, data_path), one_hot=True)
        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    def _create_placeholder(self):
        X = tf.placeholder(tf.float32, [None, 784], name="input")
        Y = tf.placeholder(tf.float32, [None, 10], name="lables")
        return X, Y 

    # input shape defaults to NHWC(batch_size, height, width, channel)
    # kernel shape defaults to (height, width, input_channel, output_channel)
    def _create_filter_layer(self, layer_name, input_layer, kernel_shape, bias_shape, stride_shape=[1, 1, 1, 1], padding='SAME'):
        with tf.variable_scope(layer_name) as scope:
            k = tf.get_variable('kernel', kernel_shape)
            b = tf.get_variable('biases', bias_shape, initializer=tf.random_normal_initializer())
            conv = tf.nn.conv2d(input_layer, k, strides=stride_shape, padding=padding)
            return tf.nn.relu(conv + b, name=scope.name)

    def _create_pool_layer(self, layer_name, input_layer, ksize, stride_shape=[1, 2, 2, 1], padding='SAME'):
        with tf.variable_scope(layer_name) as scope:
            return tf.nn.max_pool(input_layer, ksize=ksize, strides=stride_shape, padding=padding)

    def _create_fc_layer(self, layer_name, input_layer, weight_shape, bias_shape):
        with tf.variable_scope(layer_name) as scope:
            w = tf.get_variable('weights', weight_shape, initializer=tf.truncated_normal_initializer())
            b = tf.get_variable('biases', bias_shape, initializer=tf.constant_initializer(0.0))
            return tf.matmul(input_layer, w) + b

    def _create_dropout(self, input_layer):
        keep_prob = tf.placeholder(tf.float32)
        dp = tf.nn.dropout(input_layer, keep_prob)
        return dp, keep_prob


    def _create_loss(self, logits, labels):
        entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
        return tf.reduce_mean(entropy)

    def _create_accuracy(self):
        correct_preds = tf.equal(tf.argmax(self.fc2, 1), tf.argmax(self.Y, 1))
        return tf.reduce_mean(tf.cast(correct_preds, tf.float32))

    def build_graph(self):
        self.X, self.Y = self._create_placeholder()
        conv1 = self._create_filter_layer(layer_name='conv1', input_layer=tf.reshape(self.X, shape=[-1, 28, 28, 1]), kernel_shape=[5, 5, 1, 32], bias_shape=[32])
        pool1 = self._create_pool_layer(layer_name='pool1', input_layer=conv1, ksize=[1, 2, 2, 1])
        conv2 = self._create_filter_layer(layer_name='conv2', input_layer=pool1, kernel_shape=[5, 5, 32, 64], bias_shape=[64])
        pool2 = self._create_pool_layer(layer_name='pool2', input_layer=conv2, ksize=[1, 2, 2, 1])
        fc1 = self._create_fc_layer(layer_name='fc1', input_layer= tf.reshape(pool2, [-1, 7 * 7 * 64]), weight_shape=[7 * 7 * 64, 1024], bias_shape=[1024])
        dp, self.keep_prob= self._create_dropout(tf.nn.relu(fc1, name='relu'))
        self.fc2 = self._create_fc_layer(layer_name='fc2', input_layer= dp, weight_shape=[1024, 10], bias_shape=[10])
        self.loss = self._create_loss(logits=self.fc2, labels=self.Y)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        self.accuracy = self._create_accuracy()

    def train_model(self):

        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)

            for step in range(20000):
                batch = self.data.train.next_batch(self.batch_size)
                    
                if step % 100 == 0:
                    acc = self.accuracy.eval(feed_dict={self.X: batch[0], self.Y:batch[1], self.keep_prob: 1.0})
                    print("step %r accuracy=%r" % (step, acc))

                sess.run([self.optimizer], feed_dict={self.X: batch[0], self.Y:batch[1], self.keep_prob: 0.5})

            print('test accuracy %g' % self.accuracy.eval(feed_dict={self.X: self.data.test.images, self.Y: self.data.test.labels, self.keep_prob: 1.0}))
         
#model = MnistConvnet(LEARNING_RATE, BATCH_SIZE)
#model.build_graph()
#model.train_model()